In [ ]:
import os
import pandas as pd
from PIL import Image, ImageDraw
import numpy as np

# Directories
roi_dir = '/Users/daliagala/Documents/smFISH/Lac_CPTI_Kstim_ROI_cropped/'
csv_dir = '/Users/daliagala/Library/CloudStorage/OneDrive-Nexus365/Thesis/Thesis-Scripts/BigFISH_pypeline/KStim_CPTI_Lac_YFP_smFISH_Results'
path_to_save_images = '/Users/daliagala/Documents/smFISH/Lac_CPTI_Kstim_Particles/Neuron'

# Lists to hold tif and csv files
tif_files = [f for f in os.listdir(roi_dir) if f.endswith('.tif')]
csv_files = [f for f in os.listdir(csv_dir) if f.endswith('.csv')]

# Placeholder for results
results = []

for tif_file in tif_files:
    # Extracting matching csv filename
    base_name = tif_file.split('ROI_')[1].split('_C3_cropped.tif')[0]
    matching_csv = base_name + '_C4_cropped.csv'
    
    if matching_csv in csv_files:
        # Open tif image and get coordinates_1
        tif_path = os.path.join(roi_dir, tif_file)
        image = Image.open(tif_path)
        image_array = np.array(image)
        indices_1 = np.where(image_array == 255)
        coordinates_1 = list(zip(indices_1[0], indices_1[1]))
        
        # Extract list_coordinates_array from csv
        csv_path = os.path.join(csv_dir, matching_csv)
        df = pd.read_csv(csv_path, header = None)
        # Rename columns
        df.columns = ['z', 'y', 'x']

        # Drop the 'z' column
        df.drop(columns='z', inplace=True)
        list_coordinates_array = list(df[['y', 'x']].itertuples(index=False, name=None))
        
        # Get common coordinates
        common_coordinates = set(coordinates_1).intersection(set(list_coordinates_array))
        
        # Draw on the image
        draw = ImageDraw.Draw(image)
        dot_size = 2
        color = "red"
        for (y, x) in common_coordinates:
            draw.ellipse([(x - dot_size, y - dot_size), 
                          (x + dot_size, y + dot_size)], 
                          fill=color)
        
        # Save modified image
        if not os.path.exists(path_to_save_images):
            os.makedirs(path_to_save_images)
            
        output_image_path = os.path.join(path_to_save_images, 'Modified_' + tif_file)
        image.save(output_image_path)
        
        # Save results
        results.append((tif_file, len(common_coordinates)))

# Convert results to DataFrame and save as csv
df_results = pd.DataFrame(results, columns=['Filename', 'Common Coordinates'])
df_results.to_csv('final_results_KStim_Lac_YFP_smFISH_Neuron.csv', index=False)